In [1]:
import numpy as np
from scipy.spatial import distance_matrix
from matplotlib import pyplot as plt
from itertools import combinations

# 2-approximation algorithm

In [2]:
# Node
class Node:
  def __init__(self, ind, d):
    # ind: index of vertex
    # d: weight of crossing edge
    self.ind = ind
    self.d = d
    self.pi = None

In [3]:
# Min-heap Data Structure

class Heap:
  def __init__(self, a):
    # a: array of Node
    self.heap_size = len(a)
    self.array = a
    # dictionary that records the location of each vertex in heap array
    self.ind_loc = {}
    self.node_set = set()
    for i in range(self.heap_size):
      self.ind_loc[a[i].ind] = i
      self.node_set.add(i)

    return

def PARENT(i):
  return int((i-1)/2)

def LEFT(i):
  return 2*i+1

def RIGHT(i):
  return 2*i+2

def HEAPIFY(A, i):
  # A: Heap instance
  l = LEFT(i)
  r = RIGHT(i)
  if l <= A.heap_size-1 and A.array[l].d < A.array[i].d:
    minest = l
  else:
    minest = i
  if r <= A.heap_size-1 and A.array[r].d < A.array[minest].d:
    minest = r
  
  if minest != i:
    A.array[i], A.array[minest] = A.array[minest], A.array[i]
    A.ind_loc[A.array[i].ind] = i
    A.ind_loc[A.array[minest].ind] = minest
    HEAPIFY(A, minest)
  
  return

def BUILD_HEAP(a):
  # a: array
  A = Heap(a)
  for i in range(int(A.heap_size/2), -1, -1):
    HEAPIFY(A, i)
  
  return A

def HEAP_MIN(A):
  # A: Heap instance
  return A.array[0]

def EXTRACT_MIN(A):
  # A: Heap instance
  assert A.heap_size >= 1

  # min: root node
  min = A.array[0]
  A.array[0], A.array[A.heap_size-1] = A.array[A.heap_size-1], A.array[0]

  A.ind_loc[A.array[0].ind] = 0
  A.ind_loc[A.array[A.heap_size-1].ind] = A.heap_size-1

  A.node_set.remove(A.array[A.heap_size-1].ind)
  A.heap_size -= 1
  HEAPIFY(A, 0)
  return min.ind

def UPDATE_KEY(A, ind, key, pi):
  # A: Heap instance
  # ind: index of vertex

  loc = A.ind_loc[ind]

  old_key = A.array[loc].d
  if old_key <= key:
    print("Wrong Update!")
    return

  if A.array[loc].ind != ind:
    print(A.array[loc].ind, ind)  
  A.array[loc].d = key
  A.array[loc].pi = pi
      
  while loc > 0 and A.array[PARENT(loc)].d > A.array[loc].d:
    A.array[loc], A.array[PARENT(loc)] = A.array[PARENT(loc)], A.array[loc]
    A.ind_loc[A.array[loc].ind] = loc
    A.ind_loc[A.array[PARENT(loc)].ind] = PARENT(loc)
    loc = PARENT(loc)
    
  return

In [4]:
# Prim's Algorithm for finding minimal spanning tree, represented by an adjacency list

def Prim(matrix):
  # number of vertices
  N = len(matrix)
  node_array = []
  for i in range(N):
    if i == 0:
      node_array.append(Node(0, 0))
    else:
      node_array.append(Node(i, float("inf")))
  Q = BUILD_HEAP(node_array)
  while Q.heap_size != 0:
    u = EXTRACT_MIN(Q)
    for v in range(N):
      if v in Q.node_set and Q.array[Q.ind_loc[v]].d > matrix[u][v]:
        #print(u, v, Q.array[Q.ind_loc[v]].d, matrix[u][v])
        UPDATE_KEY(Q, v, matrix[u][v], u)
  
  adj_list = {}

  for i in range(N):
    u, v = Q.array[i].ind, Q.array[i].pi
    
    if u == None or v == None:
      continue

    if u in adj_list:
      if v not in adj_list[u]:
        adj_list[u].append(v)
    else:
      adj_list[u] = [v]
    
    if v in adj_list:
      if u not in adj_list[v]:
        adj_list[v].append(u)
    else:
      adj_list[v] = [u]    
  
  return Q, adj_list

In [5]:
# DFS traversal. Return an ordered list indicating the vertices as they are visited during the DFS traversal.

def DFS(G):
  # G: graph, represented as an adjacency list
  N = len(G)
  visited = [0] * N
  vertices = []
  DFS_visit(G, visited, 0, vertices)
  return vertices

def DFS_visit(G, visited, u, vertices):

  visited[u] = 1
  vertices.append(u)

  for v in G[u]:
    if visited[v] == 0:
      DFS_visit(G, visited, v, vertices)
      vertices.append(u)
  return

In [6]:
# Path with unique vertex, namely approximated TSP

def unique_vertex(vertices):
  unique_set = set()
  TSP = []
  for v in vertices:
    if v not in unique_set:
      unique_set.add(v)
      TSP.append(v)
  return TSP + [vertices[0]]

In [7]:
# All in one function

def TSP_2app(matrix):
  # matrix: distance matrix of a completed graph
  Q, adj_list = Prim(matrix)
  vertices = DFS(adj_list)

  return unique_vertex(vertices)

## Test

In [8]:
!curl -LJO https://raw.githubusercontent.com/zhangky12/TSP_RL_Q/master/wi29.tsp.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   944  100   944    0     0   4158      0 --:--:-- --:--:-- --:--:--  4158


In [9]:
f = open("wi29.tsp.txt", "r")
number = 7

In [10]:
cities_loc = []
count = 0
for x in f:
  if count == number:
    break
  if x[0].isnumeric():
    count += 1
    location = x.split(' ')
    cities_loc.append([float(location[1]), float(location[2])])

In [11]:
CG_matrix = distance_matrix(cities_loc, cities_loc)

In [12]:
app_tour = TSP_2app(CG_matrix)
app_tour

[0, 1, 5, 4, 3, 6, 2, 0]

In [ ]:
# Total length
app_length = 0
for i in range(len(app_tour)-1):
  app_length += CG_matrix[app_tour[i]][app_tour[i+1]]

app_length 

32941.47352951371

#$\frac{3}{2}$-approximation algorithm

In [8]:
# Find minimum perfect matching
def MIN_WEIGHT_MATCH(matrix, adj_list):

  odd_degree_v = set()
  match_set = []

  for i, item in enumerate(adj_list.items()):
    if len(item[1]) % 2 == 1:
      odd_degree_v.add(item[0])
  sorted_match = sorted(combinations(odd_degree_v,2), key = lambda x: matrix[x[0]][x[1]])
  sorted_match = [i for i in sorted_match if i[1] not in adj_list[i[0]]]

  n_select = int(len(odd_degree_v)/2)

  select_comb = combinations(sorted_match, n_select)

  for soln in select_comb:
    node_set = set()
    for cur_match in soln:
      node_set.add(cur_match[0])
      node_set.add(cur_match[1])
    if len(node_set) == len(odd_degree_v):
      break

  return soln

# Add matching to MST tree
def UNION_MST_MATCH(adj_list, match_set):
  for (u, v) in match_set:
    if v not in adj_list[u]:
      adj_list[u].append(v)
    if u not in adj_list[v]:
      adj_list[v].append(u)
  #return adj_list


In [9]:
# Fleury’s Algorithm for printing Eulerian Path or Circuit

import copy
# A DFS based function to count reachable vertices from v 
def DFS_count(G, u):
  visited = [0] * len(G)
  return DFS_count_helper(G, u, visited)

def DFS_count_helper(G, u, visited):
  count = 1
  visited[u] = 1
  for v in G[u]:
    if visited[v] == 0:
      count += DFS_count_helper(G, v, visited)
  
  return count

# If this edge is not a bridge, then it's safe to be visited next
def Edge_validation(G, u, v):
  if len(G[u]) == 1:
    return False
  else:
    count1 = DFS_count(G, u)
    # temporarily remove edge (u, v)
    G[u].remove(v)
    G[v].remove(u)
    count2 = DFS_count(G, u)
    # add edge (u,v) back to G
    G[u].append(v)
    G[v].append(u)

    return False if count1 > count2 else True

def Euler_tour(G, u):
  tour = []
  G_prime = copy.deepcopy(G)
  Euler_tour_helper(G, G_prime, u, tour)
  return tour

def Euler_tour_helper(G, G_prime, u, tour):

  for v in G_prime[u]:
    if Edge_validation(G, u, v):
      tour.append((u, v))
      G_prime[u].remove(v)
      G_prime[v].remove(u)
      Euler_tour_helper(G, G_prime, v, tour)
      #print(tour)

def TSP_better_app(matrix):
  Q, adj_list = Prim(CG_matrix)
  match_set = MIN_WEIGHT_MATCH(matrix, adj_list)
  UNION_MST_MATCH(adj_list, match_set)
  tour = Euler_tour(adj_list, 0)
  TSP = []
  unique_vertex = set()
  for e in tour:
    if e[0] not in unique_vertex:
      TSP.append(e[0])
      unique_vertex.add(e[0])
  TSP.append(tour[0][0])
  return TSP


## Test

In [19]:
f = open("wi29.tsp.txt", "r")
number = 25
cities_loc = []
count = 0

for x in f:
  if count == number:
    break
  if x[0].isnumeric():
    count += 1
    location = x.split(' ')
    cities_loc.append([float(location[1]), float(location[2])])

CG_matrix = distance_matrix(cities_loc, cities_loc)

In [20]:
len(CG_matrix)

25

In [21]:
better_app_tour = TSP_better_app(CG_matrix)
better_app_tour

[0,
 1,
 5,
 4,
 3,
 6,
 2,
 7,
 11,
 9,
 10,
 12,
 13,
 16,
 19,
 15,
 24,
 23,
 8,
 14,
 18,
 17,
 21,
 22,
 20,
 0]

In [22]:
# Total lenght
better_app_length = 0
for i in range(len(better_app_tour)-1):
  better_app_length += CG_matrix[better_app_tour[i]][better_app_tour[i+1]]

better_app_length 

38293.07287812476